In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import matplotlib.dates as mdates

from ydata_profiling import ProfileReport

In [ ]:
# When adding columns, add customer spending quartile from code I have in A/B testing notebook